
<center>
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> Tarea N°1, v1.00 </h1>
    
</center>



Joaquín Castillo Tapia

201773520-1 

paralelo 200


## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 30 de Abril a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).




## Introducción 

En esta primera tarea de INF/ILI-285, versión 2020-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Significancia, Errores de Cancelación y Búsqueda de Ceros. El desarrollo de cada uno de esos temas se presenta en una serie de preguntas, donde ustedes deben ir decidiendo, pregunta a pregunta, como cada uno de los temas se aplica. En general, los temas no se analizan por separado, sino de manera acoplada. Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.

In [1]:
import numpy as np
import scipy.special as sp

## Problemas

### 1. Simulador (50 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}


Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$) y la cantidad de bits que tiene en la mantisa un computador ficticio, el manejo del exponente debe considerarse de las misma forma que lo maneja _double precision_.
Esta implementación debe calcular las raíces de $f$ sin perdida de significancia con la cantidad de bits disponibles para la mantisa.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

Considere que en ese computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces.  Considere el siguiente ejemplo:

```python
# Alg. Base
a = 9819824.624837
b = 148736.523476
c = a+b

# Alg. con Representación de Punto Flotante de 'bits_mant' bits en la mantisa.
am = f_new_rep(9819824.624837,bits_mant) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant) # Aproximar el input en la nueva representación.
cm = f_new_rep(m,exp,am+bm) # Aproximar el output de la suma y cada operación en la nueva representación.
```


#### Desarrollo

**Nueva representación**

Lo primero fue construir la función para la nueva representación de los números. Se utilizaron los algoritmos  vistos en clase, en los que se trabaja la parte entera y la parte decimal por separado. Luego de separarlos se transformaron las partes en números binarios, y con el objetivo de realizar una aproximación más precisa, se calcularon tantos bits decimales como fuera posible (determinados por el mayor exponente posible y el largo de la mantisa). Una vez representado ambas partes se procedió a calcular el shift y a definir nuestra mantisa aplicando las reglas de redondeo. Finalmente, se calculó el número decimal que corresponde al número binario obtenido.

**Cálculo de raíces**

En cuanto al cálculo de las raíces del polinomio planteado, lo primero fue identificar la cantidad de raíces de $f(x)$. Al ser un polinomio de grado 3 este posee tres raíces con al menos una de ellas perteneciente a los números reales. Luego, para simplificar el cálculo se buscó alguna raíz que no dependiera de $A$ a través de la evaluación de distintos valores de $x$. Así, se logró determinar que $f(x=1)=0$. Es decir, que el polinomio presentado es factorizable por $(x-1)$, y que $1$ es una de sus raíces. 

Para factorizar $f(x)$ por $(x-1)$ se recurrió al método de división de polinomios. Se llegó al siguiente resultado:

$$(x^{3} - Ax^{2} + Ax - 1) : (x - 1) = x^{2} - (A-1)x + 1$$

Por lo tanto, se cumple la siguiente igualdad:

$$f(x) = x^{3} - Ax^{2} + Ax - 1 = (x - 1)(x^{2} - (A-1)x + 1)$$

De modo que las dos raíces faltantes son equivalentes a las raíces del polinomio $x^{2} - (A-1)x + 1$ y  al ser este un polinomio de grado $2$, sus raíces están determinadas por la siguiente fórmula:

$$ x_{1,2} = \frac{-b \pm \sqrt{b^{2} - 4ac}}{2a}$$

Al reemplazar, considerando que $a=1$, $b=-(A-1)$ y $c=1$, se obtiene la siguiente ecuación:

$$x_{1,2} = \frac{(A-1) \pm \sqrt{(A-1)^{2}-4}}{2}$$

Como vimos en el curso la resta entre dos números provoca una perdida de significancia. Por lo tanto, trataremos de evitar esta operación. En la ecuación planteada se presentan dos restas posibles, una es la que se hace dentro de la raíz cuadrada, la que es inevitable, y la resta que se genera en el denominador entre el valor de b y la raíz. Esta última es posible evitarla a través de un trabajo algebraico, pero es necesario hacer dieferencias respecto al valor de A.

Si $A>0$:

La raíz correspondiente a la suma se mantiene:

$$x_{1} = \frac{(A-1) + \sqrt{(A-1)^{2}-4}}{2}$$

En cuanto a la raíz correspondiente a la resta:

$$x_{2} = \frac{(A-1) - \sqrt{(A-1)^{2}-4}}{2}$$

multiplicamos por un uno conveniente de forma de generar una diferencia de cuadrados:

$$x_{2} = \frac{((A-1) - \sqrt{(A-1)^{2}-4})}{2} \cdot \frac{((A-1) + \sqrt{(A-1)^{2}-4})}{((A-1) + \sqrt{(A-1)^{2}-4})}$$

$$x_{2} = \frac{(A-1)^{2}- ((A-1)^2-4)}{2((A-1) + \sqrt{(A-1)^{2}-4})} = \frac{4}{2((A-1) + \sqrt{(A-1)^{2}-4})}$$

$$x_{2} = \frac{2}{(A-1) + \sqrt{(A-1)^{2}-4}}$$

Por lo tanto, para $A>0$ las soluciones son:

$$x_{1} = \frac{(A-1) + \sqrt{(A-1)^{2}-4}}{2}$$

$$ x_{2} = \frac{2}{(A-1) + \sqrt{(A-1)^{2}-4}}$$

Si $A<0$ se sigue un procedimiento análogo, obteniendose que:

$$x_{1} = \frac{(A-1) - \sqrt{(A-1)^{2}-4}}{2}$$

$$ x_{2} = \frac{2}{(A-1) - \sqrt{(A-1)^{2}-4}}$$

En conclusión, una de las raices del polinomio se da cuando $x=1$ y para encontrar las dos restantes, evitando lo más posible la pérdida de significancia, es necesario considerar el valor de $A$. 

In [2]:
"""
input:
x         : (double) número a transformar en la nueva representación
bits_mant : (int) cantidad de bits de la mantisa
-----
output:
result    : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant):
    # Algoritmo de representación de punto flotante modificada.
    #signo del numero
    if x < 0:
        signo = "-"
        x=x*(-1)
    else:
        signo = "+"
        
    #se divide en parte entera y parte decimal
  
    x_int = int(x)
    if x_int != 0:
        x_dec = float("0." + str(x).split(".")[1])
    else:
        x_dec = x
    
    bits_int = ""
    bits_dec = ""
    
    #se transforma la parte entera
    while(x_int != 0):
        bits_int = str(x_int%2) + bits_int
        x_int = int(x_int/2)
    if len(bits_int) == 0:
        bits_int = "0"
    
    #se transforma la parte decimal
    i=0
    while(x_dec != 0.0 and i < (1022+bits_mant)):
        x_dec = x_dec*2
        bits_dec = bits_dec + str(int(x_dec))
        if int(x_dec) == 1:
            x_dec = float("0." + str(x_dec).split(".")[1])

        i+=1
    if len(bits_dec) == 0:
        bits_dec = "0"
    
    #Calculo del shift y modificación de la mantisa
    if(bits_int == "0"):
        aux = bits_dec.find("1")
        if aux == -1:
            bits_dec = "0"
            shift = 0
        elif aux <= 1022:
            shift = -(aux+1)
            bits_dec = bits_dec + "0"
            bits_dec = bits_dec[aux+1:]
            bits_int = "1"
        else:
            aux = 1021
            shift = -1022
            bits_dec = bits_dec[aux+1:]
            bits_int = "0"
    else:    
        shift = len(bits_int)-1
        if shift <= 1023:
            bits_dec = bits_int[1:] + bits_dec
            bits_int= "1"
        else:
            return float(signo+"inf")

    # Redondeo de Mantisa
    if len(bits_dec) > bits_mant: 
        if bits_dec[bits_mant] == "0":
            bits_dec = bits_dec[:bits_mant]
        else:
            if "1" in bits_dec[bits_mant+1:]:
                bits_dec = bits_dec[:bits_mant]
                bits_dec = "{0:b}".format(int(bits_dec,2) + int("1",2))

            else:
                bits_dec = bits_dec[:bits_mant]
                if bits_dec[bits_mant-1] == "1":
                    bits_dec = "{0:b}".format(int(bits_dec,2) + int("1",2))

            if len(bits_dec)> bits_mant:
                bits_int = "{0:b}".format(int(bits_int,2) + int("1",2))
                bits_dec = bits_dec[1:]
                if len(bits_int) == 2:
                    bits_int = "1"
                    bits_dec = "0"
                    shift+=1
            elif len(bits_dec)< bits_mant:
                bits_dec = (bits_mant - len(bits_dec))*"0" + bits_dec

    
    # NO NECESITO EL EXP SOLO EL SHIFT
    #print(signo + bits_int + "." + bits_dec + " x 2^(",shift,")")
    
    # Transformar de binario a decimal
   
    # transformo
    dec_int = 0.0
    dec_dec = 0.0
    exp = -1
    for i in range(0,len(bits_dec)):
        if bits_dec[i] == "1":
            dec_dec += 2**(exp)
        exp = exp - 1
    exp = 0
    
    for i in range(0,len(bits_int)):
        if bits_int[-(i+1)] == "1":
            dec_int += 2**(exp)
        exp = exp + 1
    
    # se junta la parte entera con la parte decimal
    result= dec_int + dec_dec
    
    # muevo los bits según el shift 
    result = result*2**(shift)
    
    result*=float(signo+"1")
    return  result

"""
input:
A         : (double) valor A a evaluar
bits_mant : (int) cantidad de bits de la mantisa
-----
output:
x_roots   : (array) arreglo con las raíces encontradas
"""


def f_find_roots(A,bits_mant):
    # se crea una lista para almecenar las raíces encontradas
    x_roots = [1.0]

    # se transforman los inputs a la nueva representación en caso que sea necesario
    Am = f_new_rep(float(A),bits_mant)
    a = 1.0
    b = f_new_rep((Am-1.0),bits_mant)
    b = f_new_rep((b*(-1.0)),bits_mant)
    c = 1.0
    
    # se calcula por separado cada uno de las expresiones a utilizar
    exp_2a = f_new_rep((2.0*a),bits_mant)
    exp_b2 = f_new_rep((b*b),bits_mant)
    exp_4a = f_new_rep((4.0*a),bits_mant)
    exp_4ac = f_new_rep((exp_4a*c),bits_mant)
    exp_b =  f_new_rep((b*(-1.0)),bits_mant)
    exp_2c = f_new_rep((2.0*c),bits_mant)
    
    exp_b24ac = f_new_rep((exp_b2 - exp_4ac),bits_mant)
    exp_raizb24ac = f_new_rep(((exp_b24ac)**(1/2)),bits_mant)
    
    # se hace diferencia por el signo de A
    if A > 0:
        exp_braiz = f_new_rep((exp_b + exp_raizb24ac),bits_mant)
    elif A < 0:
        exp_braiz = f_new_rep((exp_b - exp_raizb24ac),bits_mant)
    
    # se calculan la raíces
    x1 = f_new_rep((exp_braiz/exp_2a),bits_mant)
    x_roots.append(x1)

    x2 = f_new_rep((exp_2c/exp_braiz),bits_mant)
    x_roots.append(x2)
    
    return np.array(x_roots)


### Polinomios de Legendre (50 puntos)
Dada la función compuesta $f$
\begin{equation}
f_{n,m}(x) = L_n(C_m(x)),
\end{equation}
donde $L_n$ es conocido como el polinomio de Legendre de grado $n$ definido de la siguiente forma:
\begin{equation}
L_{n}(x)=\frac{1}{2^{n}} \sum_{k=0}^{n}\left(\begin{array}{l}
n \\
k
\end{array}\right)^{2}(x-1)^{n-k}(x+1)^{k},
\end{equation}
y $C_m$ es el polinomio de Chebyshev
\begin{equation}
C_m(x) = \cos(m \cdot \arccos(x)).
\end{equation}

Utilizando el método de Bisección y Punto fijo se pide obtener la raíz de $f$ más cercana a $0.5$ dado un valor de $m$ y $n$



*Hint: Las raíces de Legendre son conocidas*



#### Desarrollo

**método de bisección**

Para el método de la bisección se buscaron dos raíces, la más cercana a $0.5$ hacia la derecha y la más cercana hacia la izquierda. Luego se comparó la distancia de estos valores respecto a $0.5$, retornando la menor entre ellas.

Para calcular el intervalo en cada caso se fue aumetando el rango de busqueda hasta encontrar un intervalo válido $(f(a)f(b)<0)$. En cada iteración se aumentaba el rango en $\frac{1}{10^{3}m}$, ya que a medida que aumenta *m*, disminuye la distancia entre raíces. Es decir, se busca un intervalo válido que contenga sólo una raíz.

**método de punto fijo**

En cuanto al método de punto fijo, es necesario ajustar primero la función, de modo que el algoritmo funcione. Para ello se suma un uno conveniente. Destacar que en esta ocasión el uno conveniente utilizado es tal para que $x$ tome mayor relevancia respecto a la función, pero no la suficiente para dejar de considerarla. El procedimiento es:

$$f_{n,m}(x) = 0$$

$$f_{n,m}(x) + 10^{3}x = 0 + 10^{3}x$$

$$\frac{f_{n,m}(x)}{10^{3}} + x = x$$


In [3]:
"""
input:
n     : (int) grado del polinomio de Legendre
m     : (int) grado del polinomio de Chebyshev
tol   : (double) tolerancia para la detención del algoritmo
-----
output:
root  : (double) raiz obtenida
"""
def f_Biseccion(n, m, tol):
    f = lambda x: sp.legendre(n)(sp.chebyt(m)(x))
    # primero se busca hacia la derecha
    a = 0.5
    b = 0.5
    aux = 1/((10**3)*m)
    # se calcula el largo del intevalo en donde buscar
    # se itera hasta encontrar un intervalo válido
    while f(a)*f(b) > 0:
        b += aux
    # se calcula la primera iteración del algoritmo
    c = (a+b)/2
    # se comienza la iteración
    while abs((b-a)/2) >= tol:
        if f(c) == 0:
            break
        if f(c)*f(a) < 0:
            b = c
        else:
            a = c
        c = (a+b)/2
    root_derecha = c
    
    # luego, se busca hacia la izquierda
    a = 0.5
    b = 0.5
    # se itera hasta encontrar un intervalo válido
    while f(a)*f(b) > 0:
        a -= aux
    # se calcula la primera iteración
    c = (a+b)/2
    # se itera según la tolerancia
    while abs((b-a)/2) > tol:
        if f(c) == 0:
            break
        if f(c)*f(a) < 0:
            b = c
        else:
            a = c
        c = (a+b)/2
    root_izquierda = c
    
    # se evalua cuál de las dos raíces es más cercana a 0.5
    distancia_derecha = abs(root_derecha - 0.5)
    distancia_izquierda = abs(root_izquierda - 0.5)
    if distancia_derecha > distancia_izquierda:
        root = root_izquierda
    else:
        root = root_derecha
    return root


"""
input:
n     : (int) grado del polinomio de Legendre
m     : (int) grado del polinomio de Chebyshev
tol   : (double) tolerancia para la detención del algoritmo
-----
output:
root  : (double) raiz obtenida
"""
def f_FPI(n, m, tol):
    f = lambda x: sp.legendre(n)(sp.chebyt(m)(x))/1000 + x
    # se define x_0
    x_0 = 0.5
    # se define una lista pra guardar los valores de x en cada iteración
    x = [x_0]
    while abs(x_0) > tol:
        x_0 = f(x_0)
        # se comprueba en caso de convergencia
        if x_0 - x[-1] == 0:
            break
        x.append(x_0)
    root = x[-1]
    return root

###  Determinantes (20 puntos)

Dada una matriz de dimensiones $ n \times n$ de la forma:
\begin{equation}
A
=
\begin{pmatrix}
a_{1,1} & a_{1,2}  & \dots & a_{1,n} \\
a_{2,1} & a_{2,2}  & \dots & a_{2,n} \\
\vdots &  \vdots   & \ddots & \vdots \\
a_{n,1} & a_{n,2}  & \dots & a_{n,n}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
donde $\mathbf{r}_k$ representa la $k$-ésima fila de la matriz $A$.
Considere la siguiente matriz $C_{i,j}(\alpha)$,
\begin{equation}
C_{i,j}(\alpha)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_i\,(1-\alpha)+\mathbf{r}_j\,\alpha \\
\vdots \\
\mathbf{r}_j\,(1-\alpha)+\mathbf{r}_i\,\alpha \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
de lo cual sabemos que $C_{i,j}(0)=A$ y que $C_{i,j}(1)$ es la matriz $A$ donde se intercambiaron las filas $i$ y $j$, es decir:
\begin{equation}
C_{i,j}(1)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_j \\
\vdots \\
\mathbf{r}_i \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix}.
\end{equation}
De las relaciones anteriores podemos concluir que el determinante de la matriz $A$, denominado $D=\det(A)$, es igual al determinante de $C_{i,j}(0)$, es decir $\det(C_{i,j}(0))=\det(A)=D$.
Por el otro lado, el determinante de $C_{i,j}(1)$ es $-D$, dado que es el intercambio de las filas $i$ y $j$ de la matriz $A$.
Por lo cual podemos concluir que $-D\leq \det(C_{i,j}(\alpha))\leq D$.

Utilizando el método de Bisección debe encontrar el valor de $\alpha$ con $p$ decimales de precisión que permitan que, dado los índices de las filas $i$, $j$, y $i\neq j$, el determinante de la matriz sea igual a $d$, donde $d\in[-D,D]$. 

Para esto se debe implementar una función que reciba la matriz $A$, las filas $i$ y $j$, y $p$; y retorne $\widehat{\alpha}$ tal que $\det(C_{i,j}(\widehat{\alpha}))=d$.


#### Desarrollo

Es posible transformar el ejercicio en una búsqueda de raíces al definir una función que lo permita. Por lo tanto, se define:

$$f(x) = d - \det(C_{i,j}(x))$$

En donde $C_{i,j}(x)$ es la matriz presenta en el ejercicio tal que $\alpha = x$, y $d$ es el valor del determinante buscado. De esta forma, aplicando el algoritmo de bisección, podemos resolver el problema.

In [4]:
"""
input
A: (array n x n) matriz
i: (int) índice de la fila "i".
j: (int) índice de la fila "j".
p: (int) cantidad de decimales de precision 
d: (double) valor requerido del determinante de $C_{i,j}(\alpha)$, i.e. $\det(C_{i,j}(\widehat{\alpha}))=d$.
output
alpha_hat: (double) alpha_hat tal que det(C_{i,j}(alpha_hat))=d.
"""
def find_alpha_hat(A,i,j,p,d):
    # se calcula el determinante actual de la matriz
    D = np.linalg.det(A)
    # se define el intervalo de x
    a = 0.0
    b = 1.0
    # se evalua a y b en la función. Son casos mencionados en el enunciado
    fa = d - D
    fb = d + D
    
    # casos en que el determinate búscado sea mayor al actual
    if abs(d) > abs(D):
        # se aproxima a los casos bordes mencionados en el enunciado
        if d*D < 0:
            return 1
        else:
            return 0
    
    # Algoritmo de bisección
    if fa*fb > 0:
        return "Intervalo inválido"
    # se calcula el número máximo de iteraciones
    m = np.ceil(np.log2(1/(0.5*(10**(-p)))-1))
    
    k=0
    f = 1.0 # se instancia f sólo para la primera iteración
    TOL = 10**(-p)
    
    while (abs(f) >= TOL) and (k != m):
        c = (a+b)/2
        # Se genera una copia de la matriz original para trabajar la función
        matriz_C = np.copy(A)
        f_i = np.copy(matriz_C[i])
        f_j = np.copy(matriz_C[j])
        matriz_C[i] = f_i*(1-c) + f_j*(c)
        matriz_C[j] = f_j*(1-c) + f_i*(c)
        f = d - np.linalg.det(matriz_C)
        if f == 0:
            alpha_hat = round(c,p)
            return alpha_hat
        if f*fa < 0:
            b = c
            fb = f
        else:
            a = c
            fa = f
        k+=1
    alpha_hat = round(c,p)
    return alpha_hat